In [269]:
import pandas as pd
import json
import os
import urllib3
import csv
from urllib.parse import quote
from lxml import etree

# Read the Config

In [270]:
with open('config/config.json', 'r') as infile:
    config = json.load(infile)


# Read the Inputs

In [271]:
# Read the execution plans
dfPlans = pd.read_csv('in/plans.csv', dtype='str', encoding='utf-8', na_filter=False)

# Lookup the Converted Mapping Task IDs

In [272]:
# Login to IDMC
http = urllib3.PoolManager()

data = '{ "username": "' + config['idmc']['user'] + '", "password": "' + config['idmc']['password'] + '" }'

url = 'https://' + config['idmc']['host'] + '/saas/public/core/v3/login'
r = http.request(
    'POST', 
    url,
    timeout=3000,
    body=data,
    headers={
            'Accept': 'application/json',
            'Content-Type': 'application/json'
        }
    )
    
# Convert the response into a datframe
result = json.loads(r.data.decode('utf-8'))
sessionID = result['userInfo']['sessionId']


In [273]:
# Get the secure agent ID

url = 'https://' + config['idmc']['pod'] + '.' + config['idmc']['host'] + '/saas/api/v2/runtimeEnvironment/name/' + quote(config['idmc']['agentGroupName'])
r = http.request(
    'GET', 
    url,
    timeout=3000,
    headers={
            'Accept': 'application/json',
            'icSessionId': sessionID
        }
    )
    
# Convert the response into a datframe
result = json.loads(r.data.decode('utf-8'))
agentGroupID = result['id']
agentGroupGUID = result['federatedId']
agentGroupName = config['idmc']['agentGroupName']

In [274]:
# Initialise the tasks data frame
skip = 0
limit = 200
i = 0
dfTasks = pd.DataFrame()

# Page through mapping task queries
while True:

    # Get a list of the mapping tasks
    url = 'https://' + config['idmc']['pod'] + '.' + config['idmc']['host'] + '/saas/public/core/v3/objects?q=type==%27MTT%27&limit=' + str(limit) + '&skip=' + str(skip)
    r = http.request(
        'GET', 
        url,
        timeout=3000,
        headers={
                'Accept': 'application/json',
                'INFA-SESSION-ID': sessionID
            }
        )
        
    # Convert the response into a datframe
    result = json.loads(r.data.decode('utf-8'))
    dfTmp = pd.json_normalize(result)
    dfResp = dfTmp.copy()
    taskCount = dfResp.iloc[0]['count']
    dfResp = dfResp['objects'].explode()
    dfResp = pd.DataFrame(dfResp)
    dfResp = pd.json_normalize(dfResp['objects'])
    dfTasks = pd.concat([dfTasks, dfResp], ignore_index=True)

    # Break if all records have been returned
    i = i + limit
    if i > taskCount:
        break



In [275]:
# Join the IDMC info onto the plans
dfResp['step_name'] = dfResp['path'].apply(lambda x: os.path.basename(x))
dfResp = dfResp[['step_name','id','path']]
dfResp = dfResp.rename(columns={'id': 'infa_id', 'path': 'infa_path'})
dfPlans = dfPlans.merge(dfResp, how='left', on='step_name')
dfPlans['agent_id'] = agentGroupID
dfPlans['agent_guid'] = agentGroupGUID
dfPlans['agent_name'] = agentGroupName
dfPlans['script_dir'] = config['local']['scriptsDir']
dfPlans['script_args'] = '' # TODO placeholder for any args that need to be passed to the step script

In [276]:
# Log an error if any plans did not find an existing matching task
dfMissing = dfPlans[(dfPlans['infa_id'] == '') | (dfPlans['infa_id'].isna())].copy()
if len(dfMissing.index) > 0:
    dfMissing.to_csv('out/missing_tasks.csv', index=False, quoting=csv.QUOTE_ALL)

# Read the template taskflow

In [277]:
# Define the namespace map
namespaces = {
    'aetgt': 'http://schemas.active-endpoints.com/appmodules/repository/2010/10/avrepository.xsd',
    'types1': 'http://schemas.active-endpoints.com/appmodules/repository/2010/10/avrepository.xsd'
}

# Parse the template XML file
tree = etree.parse('templates/tf_Base_Taskflow.TASKFLOW.xml')
root = tree.getroot()

# Example adding a new element

In [278]:
# Test append a new element
new_element = etree.Element("newElement")
new_element.text = "This is a new element"
parents = root.xpath("//*[local-name() = 'taskflow'][1]")

# Append the new element if found
if parents:
    for parent in parents:
        parent.append(new_element)
else:
    print("The specified parent tag was not found.")

# Test writing the xml file
tree.write('out/test.xml', pretty_print=True, xml_declaration=False, encoding='UTF-8')

[<Element {http://schemas.active-endpoints.com/appmodules/screenflow/2010/10/avosScreenflow.xsd}taskflow at 0x1d8d4a93a80>]


# Testing Below This Line

In [279]:
dfPlans.head()

,plan_wid,plan_name,plan_inactive_flag,plan_step_wid,plan_step_order,plan_step_type,step_guid,step_wid,step_cmd,step_name,infa_id,infa_path,agent_id,agent_guid,agent_name,script_dir,script_args
0,201C9CC7C59D167A79A2E247C6189A67,Echo Employee Snapshot Oracle R12.1.3,N,4F418CAB76FCD515C5A7F3C94D552CF,0,REGULAR,4F418CAB76FCD515C5A7F3C94D552CF,1911f67f35d9a487283f503fc7ab2ac,SDE_ORA_EmployeeDailySnapshotFact_2,SDE_ORA_EmployeeDailySnapshotFact_2,a0Ks8uNXYKLg38LRNzw6gv,Default/SDE_ORA_EmployeeDailySnapshotFact_2,010SU125000000000002,cPbb2XLzpoweqfBP9W6kOJ,AUW487V7S3-AAD,C:\Informatica\scripts,
1,201C9CC7C59D167A79A2E247C6189A67,Echo Employee Snapshot Oracle R12.1.3,N,D1A6418BA9B721A3282ECC5948E8866D,0,REGULAR,,15cf56791e1279889d61b836271199a,SDE_ORA_EmployeeDailySnapshotFact_3,SDE_ORA_EmployeeDailySnapshotFact_3,8gORkCCxfM8hk1VxABRRqd,Default/SDE_ORA_EmployeeDailySnapshotFact_3,010SU125000000000002,cPbb2XLzpoweqfBP9W6kOJ,AUW487V7S3-AAD,C:\Informatica\scripts,
2,201C9CC7C59D167A79A2E247C6189A67,Echo Employee Snapshot Oracle R12.1.3,N,C4FA37D5638DC49425B9A3E8572EF1,0,REGULAR,,17d561355624ced1967afc8fb7a1836e,SDE_ORA_EmployeeDailySnapshotFact_4,SDE_ORA_EmployeeDailySnapshotFact_4,6vPlMGJ7sw6juVvUtKpxYq,Default/SDE_ORA_EmployeeDailySnapshotFact_4,010SU125000000000002,cPbb2XLzpoweqfBP9W6kOJ,AUW487V7S3-AAD,C:\Informatica\scripts,
3,201C9CC7C59D167A79A2E247C6189A67,Echo Employee Snapshot Oracle R12.1.3,N,C1CB82CC89178BE3424CEE1896ED,0,REGULAR,,88818EC577E3BDEA5FAF5D56ED6E8442,,TASK_GROUP_Extract_EmployeeDailySnapshotFact_P...,7PjDO1zxryekytQ7Oh1zEP,Default/TASK_GROUP_Extract_EmployeeDailySnapsh...,010SU125000000000002,cPbb2XLzpoweqfBP9W6kOJ,AUW487V7S3-AAD,C:\Informatica\scripts,
4,201C9CC7C59D167A79A2E247C6189A67,Echo Employee Snapshot Oracle R12.1.3,N,C9EA5C489D4F8CE8B019A694E906B6B,1,REGULAR,,6bcdbdd6812a9ea517908ca566436fb,SDE_ORA_EmployeeDailySnapshotFact_1,SDE_ORA_EmployeeDailySnapshotFact_1,aeOQ6VugmCxipbYyKlDRUr,Default/SDE_ORA_EmployeeDailySnapshotFact_1,010SU125000000000002,cPbb2XLzpoweqfBP9W6kOJ,AUW487V7S3-AAD,C:\Informatica\scripts,
